In [65]:
import torch
import torch.nn as nn
import torch.nn.functional as F

<h2>nn.Module</h2>
<p>nn.Moduleはautograd依存パッケージ。forwardを絶対に含む</p>

In [66]:
#Mnist
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet,self).__init__()
        #nn.Conv2d(input_channel,output_channel,convolution_channel)
        self.conv1 = nn.Conv2d(1,6,3) #1 image input,6 channel output,3convolution
        self.conv2 = nn.Conv2d(6,16,5)
        
        #affine operation
        self.fc1 = nn.Linear(16*5*5,120) #16*5*5 = 400
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
    def forward(self,x):
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)),(2,2))
        x = x.view(-1, self.num_flat_features(x)) #viewは一つの要素の取り出し
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [67]:
#計算グラフが構築される
#Conv2dは4次元のテンソルのみをサポート : よってinputは(batch_size,channel,height,width)
net = ConvNet()
print(net)

ConvNet(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [68]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 3, 3])


In [69]:
input = torch.autograd.Variable(torch.randn(16, 1, 32, 32))
out = net(input)
#batch_sizeが16であるから16個の結果がそれぞれ10の要素を持っている。
print(out.size())

torch.Size([16, 10])


In [70]:
net.zero_grad()
out.backward(torch.randn(16, 10))

In [71]:
output = net(input)
#target =  torch.autograd.Variable(torch.arange(1, 11,dtype = torch.float))  # a dummy target, for example
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.5994, grad_fn=<MseLossBackward>)


input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> view -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss

In [72]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

<h3>zero_grad()はoptimの初期化に用いる</h3>

In [73]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0132, -0.0068,  0.0091,  0.0008,  0.0199,  0.0009])
